In [1]:
pwd

'/Users/jeremy/Desktop/nonsense/cbb_modeling'

In [2]:
import pandas as pd
import numpy as np

### Load Data

In [4]:
teams = pd.read_csv("/Users/jeremy/Desktop/nonsense/cbb_modeling/data/MTeams.csv")

In [5]:
teams.shape

(380, 4)

In [ ]:
regular_results = pd.read_csv('/Users/jeremy/Desktop/nonsense/cbb_modeling/data/MRegularSeasonDetailedResults.csv')
regular_results.shape

### Team Quality

In [ ]:
import statsmodels.api as sm

In [1]:
def prepare_data(df):
    dfswap = df[['Season', 'DayNum', 'LTeamID', 'LScore', 'WTeamID', 'WScore', 'WLoc', 'NumOT', 
    'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 
    'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']]

    dfswap.loc[df['WLoc'] == 'H', 'WLoc'] = 'A'
    dfswap.loc[df['WLoc'] == 'A', 'WLoc'] = 'H'
    df.columns.values[6] = 'location'
    dfswap.columns.values[6] = 'location'    
      
    df.columns = [x.replace('W','T1_').replace('L','T2_') for x in list(df.columns)]
    dfswap.columns = [x.replace('L','T1_').replace('W','T2_') for x in list(dfswap.columns)]

    output = pd.concat([df, dfswap]).reset_index(drop=True)
    output.loc[output.location=='N','location'] = '0'
    output.loc[output.location=='H','location'] = '1'
    output.loc[output.location=='A','location'] = '-1'
    output.location = output.location.astype(int)
    
    output['PointDiff'] = output['T1_Score'] - output['T2_Score']
    
    return output

In [2]:
regular_data = prepare_data(regular_results)

NameError: name 'regular_results' is not defined

In [28]:
regular_season_effects = regular_data[['Season','T1_TeamID','T2_TeamID','PointDiff']].copy()
regular_season_effects['T1_TeamID'] = regular_season_effects['T1_TeamID'].astype(str)
regular_season_effects['T2_TeamID'] = regular_season_effects['T2_TeamID'].astype(str)
regular_season_effects['win'] = np.where(regular_season_effects['PointDiff']>0,1,0)

In [10]:
def normalize_column(values):
  themean = np.mean(values)
  thestd = np.std(values)
  norm = (values - themean)/(thestd) 
  return(pd.DataFrame(norm))

def team_quality(season):
    formula = 'win~-1+T1_TeamID+T2_TeamID'
    glm = sm.GLM.from_formula(formula=formula, 
                              data=regular_season_effects.loc[regular_season_effects.Season==season,:], 
                              family=sm.families.Binomial()).fit()
    quality = pd.DataFrame(glm.params).reset_index()
    quality.columns = ['TeamID','quality']
    quality['Season'] = season
    quality['quality'] = normalize_column(quality['quality'])
    quality['quality'] = np.exp(quality['quality'])
    quality = quality.loc[quality.TeamID.str.contains('T1_')].reset_index(drop=True)
    quality['TeamID'] = quality['TeamID'].apply(lambda x: x[10:14]).astype(int)
    print(quality['quality'].mean(), quality['quality'].std())
    return quality

In [29]:
team_quality(2015)

8.966930743414089 129.1960868185282


,TeamID,quality,Season
0,1101,1.000000,2015
1,1102,1.713550,2015
2,1103,2.196173,2015
3,1104,2.901689,2015
4,1105,0.788499,2015
...,...,...,...
346,1460,1.410841,2015
347,1461,2.836046,2015
348,1462,3.534407,2015
349,1463,2.259003,2015


### Create features

In [ ]:
df['WPoss'] = (df['WFGA'] - df['WOR'] + df['WTO'] + (0.475*df['WFTA']) )
df['LPoss'] = (df['LFGA'] - df['LOR'] + df['LTO'] + (0.475*df['LFTA']) )
df['TotPoss'] = df['WPoss'] + df['LPoss']
df['WPPP'] = (df['WScore'] / ((df['WPoss'] + df['LPoss'])/2) )
df['LPPP'] = (df['LScore'] / ((df['WPoss'] + df['LPoss'])/2) )